In [2]:
BASE_URL = "https://ae.kickscrew.com/collections/nike"
CLASS_HITS_ITEM = "ais-Hits-item"
CLASS_HIT_IMAGE = "hit-image"
CLASS_PRODUCT_PRICE = "product-price"
CLASS_PAGINATION_LINK = "ais-Pagination-item--lastPage"


csv_file_path = "scraped_data.csv"

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv

data_list = []

try:
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    driver = webdriver.Chrome(options=options)


    wait = WebDriverWait(driver, 20)  # Adjust the timeout as needed
    driver.get(BASE_URL)
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, CLASS_PAGINATION_LINK)))


    # Find the last page number
    last_page_element = driver.find_element(By.CLASS_NAME, CLASS_PAGINATION_LINK)
    last_page_url = last_page_element.find_element(By.TAG_NAME,"a").get_attribute("href")
    last_page = int(last_page_url.split("page=")[-1])

    # Generate the list of URLs
    urls = [BASE_URL + "?page=" + str(i) for i in range(310, last_page + 1)]

    for url in urls:
        driver.get(url)

        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, CLASS_HITS_ITEM)))

        search_results = driver.find_elements(By.CLASS_NAME,CLASS_HITS_ITEM)

        for result in search_results:
            handle = result.find_element(By.TAG_NAME,"a").get_attribute("href").split("/products/")[-1]
            title = result.find_element(By.CLASS_NAME,CLASS_HIT_IMAGE).get_attribute("alt")
            price = result.find_element(By.CLASS_NAME,CLASS_PRODUCT_PRICE).text
            image_url = result.find_element(By.CLASS_NAME,CLASS_HIT_IMAGE).get_attribute("src")

            data_list.append([handle,title, price, image_url])
    
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Handle', 'Title', 'Price', 'Image URL']
        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        csv_writer.writeheader()
        csv_writer.writerows({'Handle': handle, 'Title': title, 'Price': price, 'Image URL': image_url} for handle, title, price, image_url in data_list)
        
    print(f"Data has been saved to {csv_file_path}")


finally:
    driver.quit()


Data has been saved to scraped_data.csv


In [10]:
!head -2 scraped_data.csv

Handle,Title,Price,Image URL
wmns-nike-air-force-1-low-valentine-s-day-2024-fz5068-161,(WMNS) Nike Air Force 1 Low 'Valentines Day 2024',AED 504,https://cdn.shopify.com/s/files/1/0603/3031/1875/files/main-square_57cf7b45-39ff-48f5-8ad1-e8e71f4d2e90_x480.png?v=1703145184
